In [1]:
import pandas as pd
import numpy as np

orin_df = pd.read_csv('sample_song_tag(re).csv')
orin_df = orin_df[['곡 제목', '아티스트 리스트']]
data = orin_df[['곡 제목', '아티스트 리스트']]

txt = data['아티스트 리스트'].str.replace("[", '').str.replace("]", '').str.replace("'", '')
data['아티스트 리스트'] = txt
data.columns = ['노래', '가수']
data = data[697:]
data

/var/folders/2d/k1h9bjt9385d884qrqvzk1880000gn/T/ipykernel_12993/3725806061.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  txt = data['아티스트 리스트'].str.replace("[", '').str.replace("]", '').str.replace("'", '')


,노래,가수
406,이별 앞으로,가을방학
407,San Francisco,ADOY
408,그렇습니까,흐른
409,블루진,디어클라우드
410,겨울을 걷는다 (스튜디오 라이브 버전),윤딴딴
...,...,...
1595,골고다,한지상
1596,신데렐라와 왕자의 사랑노래,극단 원공
1597,서곡_내 비극적인 이야기,"전동석, 뮤지컬 팬텀 2016 앙상블"
1598,"나의 빛, 어머니",박은태


In [14]:
import os

os.mkdir('가사')
os.mkdir('가사/1차_melon')
os.mkdir('가사/1차_melon/melon_img')

In [2]:
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import re

# 드라이버 경로 지정
driver = wd.Chrome(executable_path=r'..............................')
time.sleep(2)

# 윈도우 크기 조절
driver.maximize_window()

In [3]:
# 홈페이지 부르기
driver.get('https://www.melon.com/')

# 검색창 클릭
driver.find_element_by_xpath('//*[@id="top_search"]').click()

In [4]:
from tqdm import tqdm
import ssl
import urllib.request
from urllib.request import urlopen

lyrics_list = []
like_list = []
rank_list = []
reply_list_1 = []
reply_list_2 = []
reply_list_3 = []
reply_list_4 = []
reply_list_5 = []

for i in tqdm(range(697, 697+len(data))):
    # 가수와 노래 값 받기
    singer = data['가수'][i]
    song_name = data['노래'][i]
    
    # 검색어 창에 보내기
    all_key = song_name + ' ' + singer
    driver.find_element_by_xpath('//*[@id="top_search"]').clear()
    driver.find_element_by_xpath('//*[@id="top_search"]').send_keys(all_key)
    try:
        driver.find_element_by_xpath('//*[@id="gnb"]/fieldset/button[2]/span').click()
    except:
        driver.find_element_by_xpath('//*[@id="header_wrap"]/div[2]/fieldset/button[2]/span').click()
    # 노래 있으면 들어가기
    try:
        driver.find_element_by_xpath('//*[@id="frm_songList"]/div/table/tbody/tr/td[3]/div/div/a[1]').click()
        
        try:
            # 가사 숨겨진것 열기
            driver.find_element_by_xpath('//*[@id="lyricArea"]/button').click()
        except:
            zz=1
        # 댓글 추천 수 클릭
        driver.find_element_by_xpath('//*[@id="d_cmtpgn_cmt_count_wrapper"]/ul/li[2]/a').click()
            
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        # 가사 가져오기
        try:
            lyrics = soup.find('div', attrs={'class':'lyric on'}).text.replace('\n', ' ').replace('\t', '').replace('  ', ' ')
        except:
            lyrics = '없음'

        # 좋아요 수 가져오기
        try:
            like = soup.find('span', attrs={'class':'cnt'}).text
        except:
            like = '없음'
        
        # 순위 가져오기
        try:
            rank = soup.find('div', attrs={'class':'chart'}).text.replace('\n', '').replace('\t', '')[1:]
        except:
            rank = '없음'
        
        # 댓글 가져오기
        try:
            t = soup.find_all('div', attrs={'class':'cmt_text d_cmtpgn_cmt_full_contents'})#.text
            try:
                reply_1 = t[0].text.replace('\n', '').replace('\t', '')[5:]
            except:
                reply_1 = '없음'
            try:
                reply_2 = t[1].text.replace('\n', '').replace('\t', '')[5:]
            except:
                reply_2 = '없음'
            try:
                reply_3 = t[2].text.replace('\n', '').replace('\t', '')[5:]
            except:
                reply_3 = '없음'
            try:
                reply_4 = t[3].text.replace('\n', '').replace('\t', '')[5:]
            except:
                reply_4 = '없음'
            try:
                reply_5 = t[4].text.replace('\n', '').replace('\t', '')[5:]
            except:
                reply_5 = '없음'
        except:
            reply_1 = '없음'
            reply_2 = '없음'
            reply_3 = '없음'
            reply_4 = '없음'
            reply_5 = '없음'
               
        # 앨범 사진
        imge = driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[1]/a/img').get_attribute('src')
        context = ssl._create_unverified_context()

        with urlopen(imge, context=context) as f:
            with open('가사/1차_melon/melon_img/'+str(i)+'_'+str(singer)+'_'+str(song_name)+'.jpg', 'wb') as h:
                img = f.read()
                h.write(img)
    except:
        lyrics = '없음'
        like = '없음'
        rank = '없음'
        reply_1 = '없음'
        reply_2 = '없음'
        reply_3 = '없음'
        reply_4 = '없음'
        reply_5 = '없음'
        print('없음:', singer, song_name)
        
    lyrics_list.append(lyrics)
    like_list.append(like)
    rank_list.append(rank)
    reply_list_1.append(reply_1)
    reply_list_2.append(reply_2)
    reply_list_3.append(reply_3)
    reply_list_4.append(reply_4)
    reply_list_5.append(reply_5)

  3%|█▎                                       | 38/1194 [02:11<36:05,  1.87s/it]

없음: 도시 (DOSHi) CAMP (Feat. Brick) (Prod. Young sick)


  4%|█▊                                       | 51/1194 [02:40<30:41,  1.61s/it]

없음: Simba Zawadi (심바 자와디) 계모


  5%|██                                       | 61/1194 [02:59<30:00,  1.59s/it]

...

 19%|███████▋                                | 231/1194 [08:36<35:00,  2.18s/it]

없음: Jordan McGraw FLEXIBLE (Feat. T-Pain) (Remix)


 19%|███████▊                                | 232/1194 [08:38<35:48,  2.23s/it]

없음: Halsey I Walk The Line


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=94.0.4606.61)


In [5]:
df = pd.concat([pd.DataFrame(lyrics_list), pd.DataFrame(like_list), pd.DataFrame(rank_list),
               pd.DataFrame(reply_list_1), pd.DataFrame(reply_list_2), pd.DataFrame(reply_list_3),
               pd.DataFrame(reply_list_4), pd.DataFrame(reply_list_5)], axis=1)
df.columns = ['원가사', '좋아요수', '어제순위', '댓글1', '댓글2', '댓글3', '댓글4', '댓글5']

In [6]:
df

,원가사,좋아요수,어제순위,댓글1,댓글2,댓글3,댓글4,댓글5
0,웃어줄 수 있을 거야 거울 앞에서머리를 질끈 묶고운동화 끈을 묶고물은 충분히 미리...,"8,021",없음,전 가을방학의 그 어떤 노래 중에도 이 노래만큼 슬픈게 없어요...,헤어짐의 감정을 이토록 디테일하게 그와중에 예쁘게 그리다니 진짜 대단한 노래에요 ㅠ...,너 없이 수없이 살아야 하네,가사 속 주인공이 담담하게 이별을 준비하는 모습이 되게 울컥한 것 같아요 목소리도 ...,가사내용이랑 표현이 너무 예뻐서 더 미친듯이 슬퍼요..
1,San Francisco all the colors look backSan Fra...,"3,991",없음,댓글이 없네요. 진짜 최곤데!!,샌프란시스코 가면 걷거나 드라이브 하면서 꼭 듣고 싶은 노래,M83느낌 난다,미친.. 이런 노래를 도시에서 듣고다녀야지,샌프란시스코 골든 게이트 브릿지의 야경을 드라이브 하면서 듣고 싶은 노래
2,눈을 보고 싶은데 니 손을 잡고 싶은데너의 어깨 입술을 만지고 싶은데얘길 하고 싶...,"6,932",없음,여전히 시립다 이 노래는,좋아요,정말..좋아요,걷는 걸 좋아하는지영화를 좋아하는지어제 꿈은 어땠는지 알고 싶은데,2010년인가 고등학생 때 듣던 노랜데 여전히 좋네요
3,바다 끝없이 푸른 빛을 향해 달리네모든걸 잊은 채로파도 속에선 무거웠던 맘이 씻겨...,"3,343",없음,따듯한 푸르름을 느끼게 해주셔서당신이 음악을 만들어 주는 사람이라서노래를 들으면 세...,목요일마다 인스타 라이브 잘 듣고 있어요. 첫 싱글앨범이지요? 그래도 꽉꽉 찬 느낌...,숨가쁘게 스치는 상처들과 마주한 순간순간들 속에도 아이처럼 춤추는 자유로운 너의 오...,와.. 진짜 너무 좋다,블루진....진짜 넘나 좋은 곡........푸른노래 올여름 즐겨들을께요. 고마워요...
4,벌써 몇 달 짼가너 만난다는 그 사람 얘길 들었어아마 뭔 일이 있었나 저쨌나떠들어...,"2,344",없음,이 버전도 따듯해서 좋다... 겨울에 난로 앞에다가 손 내밀고 있는 느낌,너무 좋다,으아아아~~,최고입니다! 사르르...,이 노래를 들으며 편지를 쓰니딴딴형이 쓴 편지가 되어버렸네요 ㅋㅋ없었던 겨울에 힘들...
...,...,...,...,...,...,...,...,...
227,없음,없음,없음,없음,없음,없음,없음,없음
228,없음,없음,없음,없음,없음,없음,없음,없음
229,없음,없음,없음,없음,없음,없음,없음,없음
230,없음,없음,없음,없음,없음,없음,없음,없음


In [9]:
data.reset_index(inplace=True, drop=False)
data

,index,노래,가수
0,406,이별 앞으로,가을방학
1,407,San Francisco,ADOY
2,408,그렇습니까,흐른
3,409,블루진,디어클라우드
4,410,겨울을 걷는다 (스튜디오 라이브 버전),윤딴딴
...,...,...,...
1189,1595,골고다,한지상
1190,1596,신데렐라와 왕자의 사랑노래,극단 원공
1191,1597,서곡_내 비극적인 이야기,"전동석, 뮤지컬 팬텀 2016 앙상블"
1192,1598,"나의 빛, 어머니",박은태


In [10]:
crawled = pd.concat([data, df], axis=1)
crawled

,index,노래,가수,원가사,좋아요수,어제순위,댓글1,댓글2,댓글3,댓글4,댓글5
0,406,이별 앞으로,가을방학,웃어줄 수 있을 거야 거울 앞에서머리를 질끈 묶고운동화 끈을 묶고물은 충분히 미리...,"8,021",없음,전 가을방학의 그 어떤 노래 중에도 이 노래만큼 슬픈게 없어요...,헤어짐의 감정을 이토록 디테일하게 그와중에 예쁘게 그리다니 진짜 대단한 노래에요 ㅠ...,너 없이 수없이 살아야 하네,가사 속 주인공이 담담하게 이별을 준비하는 모습이 되게 울컥한 것 같아요 목소리도 ...,가사내용이랑 표현이 너무 예뻐서 더 미친듯이 슬퍼요..
1,407,San Francisco,ADOY,San Francisco all the colors look backSan Fra...,"3,991",없음,댓글이 없네요. 진짜 최곤데!!,샌프란시스코 가면 걷거나 드라이브 하면서 꼭 듣고 싶은 노래,M83느낌 난다,미친.. 이런 노래를 도시에서 듣고다녀야지,샌프란시스코 골든 게이트 브릿지의 야경을 드라이브 하면서 듣고 싶은 노래
2,408,그렇습니까,흐른,눈을 보고 싶은데 니 손을 잡고 싶은데너의 어깨 입술을 만지고 싶은데얘길 하고 싶...,"6,932",없음,여전히 시립다 이 노래는,좋아요,정말..좋아요,걷는 걸 좋아하는지영화를 좋아하는지어제 꿈은 어땠는지 알고 싶은데,2010년인가 고등학생 때 듣던 노랜데 여전히 좋네요
3,409,블루진,디어클라우드,바다 끝없이 푸른 빛을 향해 달리네모든걸 잊은 채로파도 속에선 무거웠던 맘이 씻겨...,"3,343",없음,따듯한 푸르름을 느끼게 해주셔서당신이 음악을 만들어 주는 사람이라서노래를 들으면 세...,목요일마다 인스타 라이브 잘 듣고 있어요. 첫 싱글앨범이지요? 그래도 꽉꽉 찬 느낌...,숨가쁘게 스치는 상처들과 마주한 순간순간들 속에도 아이처럼 춤추는 자유로운 너의 오...,와.. 진짜 너무 좋다,블루진....진짜 넘나 좋은 곡........푸른노래 올여름 즐겨들을께요. 고마워요...
4,410,겨울을 걷는다 (스튜디오 라이브 버전),윤딴딴,벌써 몇 달 짼가너 만난다는 그 사람 얘길 들었어아마 뭔 일이 있었나 저쨌나떠들어...,"2,344",없음,이 버전도 따듯해서 좋다... 겨울에 난로 앞에다가 손 내밀고 있는 느낌,너무 좋다,으아아아~~,최고입니다! 사르르...,이 노래를 들으며 편지를 쓰니딴딴형이 쓴 편지가 되어버렸네요 ㅋㅋ없었던 겨울에 힘들...
...,...,...,...,...,...,...,...,...,...,...,...
1189,1595,골고다,한지상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1190,1596,신데렐라와 왕자의 사랑노래,극단 원공,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1191,1597,서곡_내 비극적인 이야기,"전동석, 뮤지컬 팬텀 2016 앙상블",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1192,1598,"나의 빛, 어머니",박은태,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
a = crawled.dropna()
a

,index,노래,가수,원가사,좋아요수,어제순위,댓글1,댓글2,댓글3,댓글4,댓글5
0,406,이별 앞으로,가을방학,웃어줄 수 있을 거야 거울 앞에서머리를 질끈 묶고운동화 끈을 묶고물은 충분히 미리...,"8,021",없음,전 가을방학의 그 어떤 노래 중에도 이 노래만큼 슬픈게 없어요...,헤어짐의 감정을 이토록 디테일하게 그와중에 예쁘게 그리다니 진짜 대단한 노래에요 ㅠ...,너 없이 수없이 살아야 하네,가사 속 주인공이 담담하게 이별을 준비하는 모습이 되게 울컥한 것 같아요 목소리도 ...,가사내용이랑 표현이 너무 예뻐서 더 미친듯이 슬퍼요..
1,407,San Francisco,ADOY,San Francisco all the colors look backSan Fra...,"3,991",없음,댓글이 없네요. 진짜 최곤데!!,샌프란시스코 가면 걷거나 드라이브 하면서 꼭 듣고 싶은 노래,M83느낌 난다,미친.. 이런 노래를 도시에서 듣고다녀야지,샌프란시스코 골든 게이트 브릿지의 야경을 드라이브 하면서 듣고 싶은 노래
2,408,그렇습니까,흐른,눈을 보고 싶은데 니 손을 잡고 싶은데너의 어깨 입술을 만지고 싶은데얘길 하고 싶...,"6,932",없음,여전히 시립다 이 노래는,좋아요,정말..좋아요,걷는 걸 좋아하는지영화를 좋아하는지어제 꿈은 어땠는지 알고 싶은데,2010년인가 고등학생 때 듣던 노랜데 여전히 좋네요
3,409,블루진,디어클라우드,바다 끝없이 푸른 빛을 향해 달리네모든걸 잊은 채로파도 속에선 무거웠던 맘이 씻겨...,"3,343",없음,따듯한 푸르름을 느끼게 해주셔서당신이 음악을 만들어 주는 사람이라서노래를 들으면 세...,목요일마다 인스타 라이브 잘 듣고 있어요. 첫 싱글앨범이지요? 그래도 꽉꽉 찬 느낌...,숨가쁘게 스치는 상처들과 마주한 순간순간들 속에도 아이처럼 춤추는 자유로운 너의 오...,와.. 진짜 너무 좋다,블루진....진짜 넘나 좋은 곡........푸른노래 올여름 즐겨들을께요. 고마워요...
4,410,겨울을 걷는다 (스튜디오 라이브 버전),윤딴딴,벌써 몇 달 짼가너 만난다는 그 사람 얘길 들었어아마 뭔 일이 있었나 저쨌나떠들어...,"2,344",없음,이 버전도 따듯해서 좋다... 겨울에 난로 앞에다가 손 내밀고 있는 느낌,너무 좋다,으아아아~~,최고입니다! 사르르...,이 노래를 들으며 편지를 쓰니딴딴형이 쓴 편지가 되어버렸네요 ㅋㅋ없었던 겨울에 힘들...
...,...,...,...,...,...,...,...,...,...,...,...
227,633,99 Red Balloons,Goldfinger,없음,없음,없음,없음,없음,없음,없음,없음
228,634,Holla!,Mohamed Ali,없음,없음,없음,없음,없음,없음,없음,없음
229,635,Never Be Lonely,Megan Levering,없음,없음,없음,없음,없음,없음,없음,없음
230,636,FLEXIBLE (Feat. T-Pain) (Remix),Jordan McGraw,없음,없음,없음,없음,없음,없음,없음,없음


In [12]:
a.set_index('index', inplace=True)
a

,노래,가수,원가사,좋아요수,어제순위,댓글1,댓글2,댓글3,댓글4,댓글5
index,,,,,,,,,,
406,이별 앞으로,가을방학,웃어줄 수 있을 거야 거울 앞에서머리를 질끈 묶고운동화 끈을 묶고물은 충분히 미리...,"8,021",없음,전 가을방학의 그 어떤 노래 중에도 이 노래만큼 슬픈게 없어요...,헤어짐의 감정을 이토록 디테일하게 그와중에 예쁘게 그리다니 진짜 대단한 노래에요 ㅠ...,너 없이 수없이 살아야 하네,가사 속 주인공이 담담하게 이별을 준비하는 모습이 되게 울컥한 것 같아요 목소리도 ...,가사내용이랑 표현이 너무 예뻐서 더 미친듯이 슬퍼요..
407,San Francisco,ADOY,San Francisco all the colors look backSan Fra...,"3,991",없음,댓글이 없네요. 진짜 최곤데!!,샌프란시스코 가면 걷거나 드라이브 하면서 꼭 듣고 싶은 노래,M83느낌 난다,미친.. 이런 노래를 도시에서 듣고다녀야지,샌프란시스코 골든 게이트 브릿지의 야경을 드라이브 하면서 듣고 싶은 노래
408,그렇습니까,흐른,눈을 보고 싶은데 니 손을 잡고 싶은데너의 어깨 입술을 만지고 싶은데얘길 하고 싶...,"6,932",없음,여전히 시립다 이 노래는,좋아요,정말..좋아요,걷는 걸 좋아하는지영화를 좋아하는지어제 꿈은 어땠는지 알고 싶은데,2010년인가 고등학생 때 듣던 노랜데 여전히 좋네요
409,블루진,디어클라우드,바다 끝없이 푸른 빛을 향해 달리네모든걸 잊은 채로파도 속에선 무거웠던 맘이 씻겨...,"3,343",없음,따듯한 푸르름을 느끼게 해주셔서당신이 음악을 만들어 주는 사람이라서노래를 들으면 세...,목요일마다 인스타 라이브 잘 듣고 있어요. 첫 싱글앨범이지요? 그래도 꽉꽉 찬 느낌...,숨가쁘게 스치는 상처들과 마주한 순간순간들 속에도 아이처럼 춤추는 자유로운 너의 오...,와.. 진짜 너무 좋다,블루진....진짜 넘나 좋은 곡........푸른노래 올여름 즐겨들을께요. 고마워요...
410,겨울을 걷는다 (스튜디오 라이브 버전),윤딴딴,벌써 몇 달 짼가너 만난다는 그 사람 얘길 들었어아마 뭔 일이 있었나 저쨌나떠들어...,"2,344",없음,이 버전도 따듯해서 좋다... 겨울에 난로 앞에다가 손 내밀고 있는 느낌,너무 좋다,으아아아~~,최고입니다! 사르르...,이 노래를 들으며 편지를 쓰니딴딴형이 쓴 편지가 되어버렸네요 ㅋㅋ없었던 겨울에 힘들...
...,...,...,...,...,...,...,...,...,...,...
633,99 Red Balloons,Goldfinger,없음,없음,없음,없음,없음,없음,없음,없음
634,Holla!,Mohamed Ali,없음,없음,없음,없음,없음,없음,없음,없음
635,Never Be Lonely,Megan Levering,없음,없음,없음,없음,없음,없음,없음,없음


In [13]:
a.to_csv('가사/1차_melon/melon_3.csv', encoding='utf-8-sig')

In [15]:
driver.quit()